In [1]:
1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1739417241946_0005,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [2]:
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from typing import Dict
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import gc



# Initialize Spark Session
spark = SparkSession.builder \
    .appName("CombineParquetFiles") \
    .getOrCreate()

# Configuration
bucket_name = "avantcamp-partner01-prod-ingestion-bucket"
prefix = "avant/direct_mail/mailed/"
output_path = "s3://avantcamp-partner01-prod-ingestion-bucket/avant/direct_mail/combined_mailed_data/"  # Replace with your output path

# Initialize boto3 S3 client
s3 = boto3.client("s3")

# Define the schema for writing
schema = StructType([
    StructField("contact_urn", StringType(), True),
    StructField("perm_id", StringType(), True),
    StructField("tu_perm_id", StringType(), True),
    StructField("indiv_id", LongType(), True),
    StructField("hh_id", LongType(), True),
    StructField("first_name", StringType(), True),
    StructField("middle_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("unit_type", StringType(), True),
    StructField("unit_number", StringType(), True),
    StructField("city_name", StringType(), True),
    StructField("state", StringType(), True),
    StructField("timesmailed", StringType(), True),
    StructField("timesmailed_product", StringType(), True),
    StructField("poc", StringType(), True),
    StructField("creative_type", StringType(), True),
    StructField("a_score", IntegerType(), True),
    StructField("b_score", IntegerType(), True),
    StructField("c_score", IntegerType(), True),
    StructField("zip_code", StringType(), True),
    StructField("zip4_code", StringType(), True),
    StructField("dob", StringType(), True),
    StructField("ssn8", StringType(), True),
    StructField("external_model_2_score", IntegerType(), True),
    StructField("edie04", IntegerType(), True),
    StructField("attr08", IntegerType(), True),
    StructField("lien1", IntegerType(), True),
    StructField("judg1", IntegerType(), True),
    StructField("pub1", IntegerType(), True),
    StructField("cctrd1", IntegerType(), True),
    StructField("stud01", IntegerType(), True),
    StructField("stud02", IntegerType(), True),
    StructField("lien3", IntegerType(), True),
    StructField("judg3", IntegerType(), True),
    StructField("at01", IntegerType(), True),
    StructField("at03", IntegerType(), True),
    StructField("at05", IntegerType(), True),
    StructField("at06", IntegerType(), True),
    StructField("at07", IntegerType(), True),
    StructField("at08", IntegerType(), True),
    StructField("at09", IntegerType(), True),
    StructField("at10", IntegerType(), True),
    StructField("at11", IntegerType(), True),
    StructField("at12", IntegerType(), True),
    StructField("at13", IntegerType(), True),
    StructField("at14", IntegerType(), True),
    StructField("at20", IntegerType(), True),
    StructField("at21", IntegerType(), True),
    StructField("at23", IntegerType(), True),
    StructField("at24", IntegerType(), True),
    StructField("at25", IntegerType(), True),
    StructField("at26", IntegerType(), True),
    StructField("at27", IntegerType(), True),
    StructField("at28", IntegerType(), True),
    StructField("at29", IntegerType(), True),
    StructField("at33", IntegerType(), True),
    StructField("at34", DecimalType(10, 2), True),
    StructField("at35", IntegerType(), True),
    StructField("at36", IntegerType(), True),
    StructField("at99", IntegerType(), True),
    StructField("bc01", IntegerType(), True),
    StructField("bc02", IntegerType(), True),
    StructField("bc03", IntegerType(), True),
    StructField("bc05", IntegerType(), True),
    StructField("bc06", IntegerType(), True),
    StructField("bc07", IntegerType(), True),
    StructField("bc08", IntegerType(), True),
    StructField("bc09", IntegerType(), True),
    StructField("bc10", IntegerType(), True),
    StructField("bc11", IntegerType(), True),
    StructField("bc12", IntegerType(), True),
    StructField("bc13", IntegerType(), True),
    StructField("bc14", IntegerType(), True),
    StructField("bc21", IntegerType(), True),
    StructField("bc29", IntegerType(), True),
    StructField("bc30", DecimalType(10, 2), True),
    StructField("bc31", DecimalType(10, 2), True),
    StructField("bc34", DecimalType(10, 2), True),
    StructField("bc35", IntegerType(), True),
    StructField("bc36", IntegerType(), True),
    StructField("bc98", IntegerType(), True),
    StructField("bi01", IntegerType(), True),
    StructField("bi03", IntegerType(), True),
    StructField("bi05", IntegerType(), True),
    StructField("bi06", IntegerType(), True),
    StructField("bi07", IntegerType(), True),
    StructField("bi08", IntegerType(), True),
    StructField("bi09", IntegerType(), True),
    StructField("bi20", IntegerType(), True),
    StructField("bi28", IntegerType(), True),
    StructField("br03", IntegerType(), True),
    StructField("br20", IntegerType(), True),
    StructField("br28", IntegerType(), True),
    StructField("br33", IntegerType(), True),
    StructField("ds02", IntegerType(), True),
    StructField("ds03", IntegerType(), True),
    StructField("ds04", IntegerType(), True),
    StructField("ds05", IntegerType(), True),
    StructField("ds06", IntegerType(), True),
    StructField("ds07", IntegerType(), True),
    StructField("ds08", IntegerType(), True),
    StructField("ds09", IntegerType(), True),
    StructField("ds10", IntegerType(), True),
    StructField("ds11", IntegerType(), True),
    StructField("ds12", IntegerType(), True),
    StructField("ds13", IntegerType(), True),
    StructField("ds14", IntegerType(), True),
    StructField("ds21", IntegerType(), True),
    StructField("ds33", IntegerType(), True),
    StructField("ds35", IntegerType(), True),
    StructField("fi01", IntegerType(), True),
    StructField("fi03", IntegerType(), True),
    StructField("fi05", IntegerType(), True),
    StructField("fi06", IntegerType(), True),
    StructField("fi07", IntegerType(), True),
    StructField("fi08", IntegerType(), True),
    StructField("fi09", IntegerType(), True),
    StructField("fr03", IntegerType(), True),
    StructField("fr33", IntegerType(), True),
    StructField("fr35", IntegerType(), True),
    StructField("g001", IntegerType(), True),
    StructField("g002", IntegerType(), True),
    StructField("g003", IntegerType(), True),
    StructField("g004", IntegerType(), True),
    StructField("g005", IntegerType(), True),
    StructField("g006", IntegerType(), True),
    StructField("g007", IntegerType(), True),
    StructField("g008", IntegerType(), True),
    StructField("g009", IntegerType(), True),
    StructField("g016", IntegerType(), True),
    StructField("g017", IntegerType(), True),
    StructField("g018", IntegerType(), True),
    StructField("g019", IntegerType(), True),
    StructField("g020", IntegerType(), True),
    StructField("g021", IntegerType(), True),
    StructField("g022", IntegerType(), True),
    StructField("g023", IntegerType(), True),
    StructField("g024", IntegerType(), True),
    StructField("g025", IntegerType(), True),
    StructField("g026", IntegerType(), True),
    StructField("g027", IntegerType(), True),
    StructField("g028", IntegerType(), True),
    StructField("g029", IntegerType(), True),
    StructField("g030", IntegerType(), True),
    StructField("g041", IntegerType(), True),
    StructField("g042", IntegerType(), True),
    StructField("g043", IntegerType(), True),
    StructField("g044", IntegerType(), True),
    StructField("g045", IntegerType(), True),
    StructField("g046", IntegerType(), True),
    StructField("g047", IntegerType(), True),
    StructField("g048", IntegerType(), True),
    StructField("g049", IntegerType(), True),
    StructField("g050", IntegerType(), True),
    StructField("g051", DecimalType(10, 2), True),
    StructField("g057", IntegerType(), True),
    StructField("g058", IntegerType(), True),
    StructField("g059", IntegerType(), True),
    StructField("g060", IntegerType(), True),
    StructField("g061", IntegerType(), True),
    StructField("g062", IntegerType(), True),
    StructField("g063", IntegerType(), True),
    StructField("g064", IntegerType(), True),
    StructField("g065", IntegerType(), True),
    StructField("g066", IntegerType(), True),
    StructField("g067", IntegerType(), True),
    StructField("g068", IntegerType(), True),
    StructField("g069", IntegerType(), True),
    StructField("g070", IntegerType(), True),
    StructField("g071", IntegerType(), True),
    StructField("g082", IntegerType(), True),
    StructField("g083", IntegerType(), True),
    StructField("g084", IntegerType(), True),
    StructField("g085", IntegerType(), True),
    StructField("g086", IntegerType(), True),
    StructField("g087", IntegerType(), True),
    StructField("g088", IntegerType(), True),
    StructField("g089", IntegerType(), True),
    StructField("g091", IntegerType(), True),
    StructField("g093", IntegerType(), True),
    StructField("g094", IntegerType(), True),
    StructField("g095", IntegerType(), True),
    StructField("g096", IntegerType(), True),
    StructField("g098", IntegerType(), True),
    StructField("g102", IntegerType(), True),
    StructField("g103", IntegerType(), True),
    StructField("g104", IntegerType(), True),
    StructField("in03", IntegerType(), True),
    StructField("in05", IntegerType(), True),
    StructField("in06", IntegerType(), True),
    StructField("in07", IntegerType(), True),
    StructField("in08", IntegerType(), True),
    StructField("in09", IntegerType(), True),
    StructField("in10", IntegerType(), True),
    StructField("in11", IntegerType(), True),
    StructField("in12", IntegerType(), True),
    StructField("in13", IntegerType(), True),
    StructField("in14", IntegerType(), True),
    StructField("in21", IntegerType(), True),
    StructField("in28", IntegerType(), True),
    StructField("in33", IntegerType(), True),
    StructField("in34", DecimalType(10, 2), True),
    StructField("mt01", IntegerType(), True),
    StructField("mt02", IntegerType(), True),
    StructField("mt03", IntegerType(), True),
    StructField("mt04", IntegerType(), True),
    StructField("mt20", IntegerType(), True),
    StructField("mt21", IntegerType(), True),
    StructField("mt22", IntegerType(), True),
    StructField("mt28", IntegerType(), True),
    StructField("mt29", IntegerType(), True),
    StructField("mt32", IntegerType(), True),
    StructField("mt33", IntegerType(), True),
    StructField("mt34", DecimalType(10, 2), True),
    StructField("mt35", IntegerType(), True),
    StructField("mt36", IntegerType(), True),
    StructField("mt41", IntegerType(), True),
    StructField("mt42", IntegerType(), True),
    StructField("mt43", IntegerType(), True),
    StructField("mt44", IntegerType(), True),
    StructField("mt45", IntegerType(), True),
    StructField("mt46", IntegerType(), True),
    StructField("mt47", IntegerType(), True),
    StructField("mt48", IntegerType(), True),
    StructField("mt49", IntegerType(), True),
    StructField("mt50", IntegerType(), True),
    StructField("mt51", IntegerType(), True),
    StructField("mt52", IntegerType(), True),
    StructField("mt53", IntegerType(), True),
    StructField("mt54", IntegerType(), True),
    StructField("mt55", IntegerType(), True),
    StructField("mt56", IntegerType(), True),
    StructField("mt57", IntegerType(), True),
    StructField("of01", IntegerType(), True),
    StructField("of03", IntegerType(), True),
    StructField("of20", IntegerType(), True),
    StructField("of28", IntegerType(), True),
    StructField("of29", IntegerType(), True),
    StructField("of33", IntegerType(), True),
    StructField("of36", IntegerType(), True),
    StructField("on01", IntegerType(), True),
    StructField("on03", IntegerType(), True),
    StructField("on20", IntegerType(), True),
    StructField("on33", IntegerType(), True),
    StructField("on34", DecimalType(10, 2), True),
    StructField("pb03", IntegerType(), True),
    StructField("pb05", IntegerType(), True),
    StructField("pb06", IntegerType(), True),
    StructField("pb07", IntegerType(), True),
    StructField("pb08", IntegerType(), True),
    StructField("pb09", IntegerType(), True),
    StructField("pb10", IntegerType(), True),
    StructField("pb11", IntegerType(), True),
    StructField("pb12", IntegerType(), True),
    StructField("pb13", IntegerType(), True),
    StructField("pb14", IntegerType(), True),
    StructField("pb20", IntegerType(), True),
    StructField("pb21", IntegerType(), True),
    StructField("pb33", IntegerType(), True),
    StructField("pb35", IntegerType(), True),
    StructField("pf02", IntegerType(), True),
    StructField("pf03", IntegerType(), True),
    StructField("pf05", IntegerType(), True),
    StructField("pf06", IntegerType(), True),
    StructField("pf07", IntegerType(), True),
    StructField("pf08", IntegerType(), True),
    StructField("pf09", IntegerType(), True),
    StructField("pf33", IntegerType(), True),
    StructField("pf34", DecimalType(10, 2), True),
    StructField("re03", IntegerType(), True),
    StructField("re10", IntegerType(), True),
    StructField("re11", IntegerType(), True),
    StructField("re12", IntegerType(), True),
    StructField("re13", IntegerType(), True),
    StructField("re14", IntegerType(), True),
    StructField("re20", IntegerType(), True),
    StructField("re28", IntegerType(), True),
    StructField("re32", IntegerType(), True),
    StructField("re33", IntegerType(), True),
    StructField("re34", DecimalType(10, 2), True),
    StructField("re35", IntegerType(), True),
    StructField("rt01", IntegerType(), True),
    StructField("rt03", IntegerType(), True),
    StructField("rt05", IntegerType(), True),
    StructField("rt06", IntegerType(), True),
    StructField("rt07", IntegerType(), True),
    StructField("rt08", IntegerType(), True),
    StructField("rt09", IntegerType(), True),
    StructField("rt10", IntegerType(), True),
    StructField("rt11", IntegerType(), True),
    StructField("rt12", IntegerType(), True),
    StructField("rt13", IntegerType(), True),
    StructField("rt14", IntegerType(), True),
    StructField("rt20", IntegerType(), True),
    StructField("rt21", IntegerType(), True),
    StructField("rt28", IntegerType(), True),
    StructField("rt29", IntegerType(), True),
    StructField("rt33", IntegerType(), True),
    StructField("rt34", DecimalType(10, 2), True),
    StructField("rt35", IntegerType(), True),
    StructField("rt36", IntegerType(), True),
    StructField("s002", IntegerType(), True),
    StructField("s004", IntegerType(), True),
    StructField("s008", IntegerType(), True),
    StructField("s009", IntegerType(), True),
    StructField("s010", IntegerType(), True),
    StructField("s011", IntegerType(), True),
    StructField("s012", IntegerType(), True),
    StructField("s014", IntegerType(), True),
    StructField("s015", IntegerType(), True),
    StructField("s018", IntegerType(), True),
    StructField("s019", IntegerType(), True),
    StructField("s020", IntegerType(), True),
    StructField("s027", IntegerType(), True),
    StructField("s040", IntegerType(), True),
    StructField("s043", IntegerType(), True),
    StructField("s046", DecimalType(10, 2), True),
    StructField("s054", IntegerType(), True),
    StructField("s055", IntegerType(), True),
    StructField("s059", IntegerType(), True),
    StructField("s060", IntegerType(), True),
    StructField("s061", IntegerType(), True),
    StructField("s062", IntegerType(), True),
    StructField("s063", IntegerType(), True),
    StructField("s064", IntegerType(), True),
    StructField("s065", IntegerType(), True),
    StructField("s066", IntegerType(), True),
    StructField("s078", DecimalType(10, 2), True),
    StructField("s079", DecimalType(10, 2), True),
    StructField("s0y2", IntegerType(), True),
    StructField("s114", IntegerType(), True),
    StructField("s115", IntegerType(), True),
    StructField("ur03", IntegerType(), True),
    StructField("ur05", IntegerType(), True),
    StructField("ur06", IntegerType(), True),
    StructField("ur07", IntegerType(), True),
    StructField("ur08", IntegerType(), True),
    StructField("ur09", IntegerType(), True),
    StructField("ur10", IntegerType(), True),
    StructField("ur11", IntegerType(), True),
    StructField("ur12", IntegerType(), True),
    StructField("ur13", IntegerType(), True),
    StructField("ur14", IntegerType(), True),
    StructField("ur20", IntegerType(), True),
    StructField("ur21", IntegerType(), True),
    StructField("ur28", IntegerType(), True),
    StructField("ur33", IntegerType(), True),
    StructField("ur35", IntegerType(), True),
    StructField("deadflag", StringType(), True),
    StructField("promosup", StringType(), True),
    StructField("lockflag", StringType(), True),
    StructField("sffxflag", StringType(), True),
    StructField("rr_flag", StringType(), True),
    StructField("pob_flag", StringType(), True),
    StructField("gen_flag", StringType(), True),
    StructField("miliflag", StringType(), True),
    StructField("curaddr", IntegerType(), True),
    StructField("rmdspflg", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("infrage", IntegerType(), True),
    StructField("aka_flag", StringType(), True),
    StructField("rundate", IntegerType(), True),
    StructField("hmequity", StringType(), True),
    StructField("homeflag", StringType(), True),
    StructField("cstmflg", StringType(), True),
    StructField("hawkflag", StringType(), True),
    StructField("majdrgi", StringType(), True),
    StructField("majdrgh", StringType(), True),
    StructField("mindrgi", StringType(), True),
    StructField("mindrgh", StringType(), True),
    StructField("studact", StringType(), True),
    StructField("studdef", StringType(), True),
    StructField("derogpub", IntegerType(), True),
    StructField("bankrpt", IntegerType(), True),
    StructField("bankin12", IntegerType(), True),
    StructField("bankin24", IntegerType(), True),
    StructField("agbkrupt", IntegerType(), True),
    StructField("foreclth", IntegerType(), True),
    StructField("forecl12", IntegerType(), True),
    StructField("forecl24", IntegerType(), True),
    StructField("lienth", IntegerType(), True),
    StructField("lien12", IntegerType(), True),
    StructField("lien24", IntegerType(), True),
    StructField("rellien", IntegerType(), True),
    StructField("satlien", IntegerType(), True),
    StructField("chrgoff", IntegerType(), True),
    StructField("chgoff12", IntegerType(), True),
    StructField("chgoff24", IntegerType(), True),
    StructField("agcobal", IntegerType(), True),
    StructField("repoth", IntegerType(), True),
    StructField("repo12", IntegerType(), True),
    StructField("repo24", IntegerType(), True),
    StructField("judgsuit", IntegerType(), True),
    StructField("jdsuit12", IntegerType(), True),
    StructField("trustpr", IntegerType(), True),
    StructField("numpubrc", IntegerType(), True),
    StructField("dpbrec12", IntegerType(), True),
    StructField("dpbrec24", IntegerType(), True),
    StructField("dpbrec36", IntegerType(), True),
    StructField("dtaxlien", IntegerType(), True),
    StructField("agpubrec", IntegerType(), True),
    StructField("agpub36", IntegerType(), True),
    StructField("disbnkth", IntegerType(), True),
    StructField("ndisbank", IntegerType(), True),
    StructField("bankpub6", IntegerType(), True),
    StructField("bankrp60", IntegerType(), True),
    StructField("paidjudg", IntegerType(), True),
    StructField("unpdjudg", IntegerType(), True),
    StructField("allcollc", IntegerType(), True),
    StructField("colexmed", IntegerType(), True),
    StructField("collec6", IntegerType(), True),
    StructField("colexmd6", IntegerType(), True),
    StructField("collec12", IntegerType(), True),
    StructField("colxmd12", IntegerType(), True),
    StructField("colnm12", IntegerType(), True),
    StructField("collec24", IntegerType(), True),
    StructField("colxmd24", IntegerType(), True),
    StructField("colnm24", IntegerType(), True),
    StructField("collec36", IntegerType(), True),
    StructField("colxmd36", IntegerType(), True),
    StructField("colamt24", IntegerType(), True),
    StructField("colamt6", IntegerType(), True),
    StructField("colamt12", IntegerType(), True),
    StructField("colamt36", IntegerType(), True),
    StructField("colamtth", IntegerType(), True),
    StructField("colbal24", IntegerType(), True),
    StructField("colbal6", IntegerType(), True),
    StructField("colbal12", IntegerType(), True),
    StructField("colbal36", IntegerType(), True),
    StructField("colbalth", IntegerType(), True),
    StructField("paidcol", IntegerType(), True),
    StructField("unpdcol", IntegerType(), True),
    StructField("agcoltha", IntegerType(), True),
    StructField("agcolthb", IntegerType(), True),
    StructField("trdmonop", IntegerType(), True),
    StructField("trdmof", IntegerType(), True),
    StructField("trdcnt", IntegerType(), True),
    StructField("trdsat", IntegerType(), True),
    StructField("trdopn", IntegerType(), True),
    StructField("trdver3", IntegerType(), True),
    StructField("trdver6", IntegerType(), True),
    StructField("trdver12", IntegerType(), True),
    StructField("trdver24", IntegerType(), True),
    StructField("trdbal", IntegerType(), True),
    StructField("trdbalm", IntegerType(), True),
    StructField("trdhghbl", IntegerType(), True),
    StructField("trdbal6", IntegerType(), True),
    StructField("trdhc6", IntegerType(), True),
    StructField("trdcl6", IntegerType(), True),
    StructField("trdutil", IntegerType(), True),
    StructField("trdutlop", IntegerType(), True),
    StructField("trd0bal", IntegerType(), True),
    StructField("auttrds", IntegerType(), True),
    StructField("automof", IntegerType(), True),
    StructField("autopv6", IntegerType(), True),
    StructField("autopn6", IntegerType(), True),
    StructField("autbalt", IntegerType(), True),
    StructField("autnobal", IntegerType(), True),
    StructField("autoutil", IntegerType(), True),
    StructField("autnutil", IntegerType(), True),
    StructField("rettrd", IntegerType(), True),
    StructField("retmon", IntegerType(), True),
    StructField("retmnold", IntegerType(), True),
    StructField("retopn12", IntegerType(), True),
    StructField("retver12", IntegerType(), True),
    StructField("retopen", IntegerType(), True),
    StructField("numdept", IntegerType(), True),
    StructField("retbal12", IntegerType(), True),
    StructField("rettot12", IntegerType(), True),
    StructField("rethibal", IntegerType(), True),
    StructField("rettotcl", IntegerType(), True),
    StructField("rettothc", IntegerType(), True),
    StructField("retutil", IntegerType(), True),
    StructField("rethccl", IntegerType(), True),
    StructField("retrevth", IntegerType(), True),
    StructField("retrev6", IntegerType(), True),
    StructField("retrev12", IntegerType(), True),
    StructField("fioact3", IntegerType(), True),
    StructField("fioactth", IntegerType(), True),
    StructField("fiopenth", IntegerType(), True),
    StructField("fiopblth", IntegerType(), True),
    StructField("fiagblth", IntegerType(), True),
    StructField("firatio", IntegerType(), True),
    StructField("firatoth", IntegerType(), True),
    StructField("fiopenmo", IntegerType(), True),
    StructField("fitrmof", IntegerType(), True),
    StructField("numsattr", IntegerType(), True),
    StructField("revopn6", IntegerType(), True),
    StructField("rev12mth", IntegerType(), True),
    StructField("revtotbl", IntegerType(), True),
    StructField("revaggcr", IntegerType(), True),
    StructField("revratio", IntegerType(), True),
    StructField("revmof", IntegerType(), True),
    StructField("bkrver6", IntegerType(), True),
    StructField("bkrovr6", IntegerType(), True),
    StructField("bkropen", IntegerType(), True),
    StructField("bkropn6", IntegerType(), True),
    StructField("bkropn12", IntegerType(), True),
    StructField("bkrtrds", IntegerType(), True),
    StructField("bkroage", IntegerType(), True),
    StructField("bkrnage", IntegerType(), True),
    StructField("bkrtbal", IntegerType(), True),
    StructField("bkrcrdlt", IntegerType(), True),
    StructField("bkrhbal", IntegerType(), True),
    StructField("bkrnobal", IntegerType(), True),
    StructField("bkrutil", IntegerType(), True),
    StructField("bknage", IntegerType(), True),
    StructField("bkopenth", IntegerType(), True),
    StructField("bkopenv6", IntegerType(), True),
    StructField("bkopen12", IntegerType(), True),
    StructField("bkopbal", IntegerType(), True),
    StructField("bkopbal6", IntegerType(), True),
    StructField("bkopbl12", IntegerType(), True),
    StructField("bkratth", IntegerType(), True),
    StructField("bktrades", IntegerType(), True),
    StructField("bkagblth", IntegerType(), True),
    StructField("bkagbal", IntegerType(), True),
    StructField("bkhicrlm", IntegerType(), True),
    StructField("bkbal", IntegerType(), True),
    StructField("bkutil", IntegerType(), True),
    StructField("bankmof", IntegerType(), True),
    StructField("bkitnum", IntegerType(), True),
    StructField("bkibal", IntegerType(), True),
    StructField("bkiutil", IntegerType(), True),
    StructField("bkimof", IntegerType(), True),
    StructField("instrd", IntegerType(), True),
    StructField("instrbal", IntegerType(), True),
    StructField("insutil", IntegerType(), True),
    StructField("instmof", IntegerType(), True),
    StructField("cmop2", IntegerType(), True),
    StructField("cmop2up", IntegerType(), True),
    StructField("cmop3", IntegerType(), True),
    StructField("cmop3up", IntegerType(), True),
    StructField("cmop4", IntegerType(), True),
    StructField("cmop4up", IntegerType(), True),
    StructField("cmop5", IntegerType(), True),
    StructField("cmop5up", IntegerType(), True),
    StructField("cmop7", IntegerType(), True),
    StructField("cmop7up", IntegerType(), True),
    StructField("hmop2th", IntegerType(), True),
    StructField("hmop2gth", IntegerType(), True),
    StructField("hmop3th", IntegerType(), True),
    StructField("hmop3gth", IntegerType(), True),
    StructField("hmop4th", IntegerType(), True),
    StructField("hmop4gth", IntegerType(), True),
    StructField("hmop5th", IntegerType(), True),
    StructField("hmop5gth", IntegerType(), True),
    StructField("cmop8", IntegerType(), True),
    StructField("cmop9", IntegerType(), True),
    StructField("hmp2n6", IntegerType(), True),
    StructField("hmp2gn6", IntegerType(), True),
    StructField("hmp2n12", IntegerType(), True),
    StructField("hmp2gn12", IntegerType(), True),
    StructField("hmp2n24", IntegerType(), True),
    StructField("hmp2gn24", IntegerType(), True),
    StructField("hmp3n6", IntegerType(), True),
    StructField("hmp3gn6", IntegerType(), True),
    StructField("hmp3n12", IntegerType(), True),
    StructField("hmp3gn12", IntegerType(), True),
    StructField("hmp3n24", IntegerType(), True),
    StructField("hmp3gn24", IntegerType(), True),
    StructField("hmp4n6", IntegerType(), True),
    StructField("hmp4gn6", IntegerType(), True),
    StructField("hmp4n12", IntegerType(), True),
    StructField("hmp4gn12", IntegerType(), True),
    StructField("hmp4n24", IntegerType(), True),
    StructField("hmp4gn24", IntegerType(), True),
    StructField("hmp5n6", IntegerType(), True),
    StructField("hmp5gn6", IntegerType(), True),
    StructField("hmp5n12", IntegerType(), True),
    StructField("hmp5gn12", IntegerType(), True),
    StructField("hmp5n24", IntegerType(), True),
    StructField("hmp5gn24", IntegerType(), True),
    StructField("h2in6af", IntegerType(), True),
    StructField("h2gin6af", IntegerType(), True),
    StructField("h2in12af", IntegerType(), True),
    StructField("h2gi12af", IntegerType(), True),
    StructField("h2in24af", IntegerType(), True),
    StructField("h2gi24af", IntegerType(), True),
    StructField("h3in6af", IntegerType(), True),
    StructField("h3gin6af", IntegerType(), True),
    StructField("h3in12af", IntegerType(), True),
    StructField("h3gi12af", IntegerType(), True),
    StructField("h3in24af", IntegerType(), True),
    StructField("h3gi24af", IntegerType(), True),
    StructField("h4in6af", IntegerType(), True),
    StructField("h4gin6af", IntegerType(), True),
    StructField("h4in12af", IntegerType(), True),
    StructField("h4gi12af", IntegerType(), True),
    StructField("h4in24af", IntegerType(), True),
    StructField("h4gi24af", IntegerType(), True),
    StructField("h5in6af", IntegerType(), True),
    StructField("h5gin6af", IntegerType(), True),
    StructField("h5in12af", IntegerType(), True),
    StructField("h5gi12af", IntegerType(), True),
    StructField("h5in24af", IntegerType(), True),
    StructField("h5gi24af", IntegerType(), True),
    StructField("cmp02in6", IntegerType(), True),
    StructField("cmp03in6", IntegerType(), True),
    StructField("cmp04in6", IntegerType(), True),
    StructField("cmp05in6", IntegerType(), True),
    StructField("cur2in12", IntegerType(), True),
    StructField("cur2gn12", IntegerType(), True),
    StructField("cur3in12", IntegerType(), True),
    StructField("cur3gn12", IntegerType(), True),
    StructField("cur4in12", IntegerType(), True),
    StructField("cur4gn12", IntegerType(), True),
    StructField("cur5in12", IntegerType(), True),
    StructField("cur5gn12", IntegerType(), True),
    StructField("c2in24", IntegerType(), True),
    StructField("c2gin24", IntegerType(), True),
    StructField("c3in24", IntegerType(), True),
    StructField("c3gin24", IntegerType(), True),
    StructField("c4in24", IntegerType(), True),
    StructField("c4gin24", IntegerType(), True),
    StructField("c5in24", IntegerType(), True),
    StructField("c5gin24", IntegerType(), True),
    StructField("numinq3", IntegerType(), True),
    StructField("numinq6", IntegerType(), True),
    StructField("numinq12", IntegerType(), True),
    StructField("numinq", IntegerType(), True),
    StructField("inqnoau3", IntegerType(), True),
    StructField("inqnoau6", IntegerType(), True),
    StructField("inqnoa12", IntegerType(), True),
    StructField("inqnoa", IntegerType(), True),
    StructField("recntinq", IntegerType(), True),
    StructField("ageinq", IntegerType(), True),
    StructField("fincinq", IntegerType(), True),
    StructField("mortinq", IntegerType(), True),
    StructField("mortinq6", IntegerType(), True),
    StructField("bankinq", IntegerType(), True),
    StructField("bankinq6", IntegerType(), True),
    StructField("bnkinq12", IntegerType(), True),
    StructField("colinq6", IntegerType(), True),
    StructField("colinq12", IntegerType(), True),
    StructField("colinq3", IntegerType(), True),
    StructField("colinq", IntegerType(), True),
    StructField("mortgnum", IntegerType(), True),
    StructField("mortmof", IntegerType(), True),
    StructField("mortin6", IntegerType(), True),
    StructField("mortin12", IntegerType(), True),
    StructField("morthbal", IntegerType(), True),
    StructField("mrtagbal", IntegerType(), True),
    StructField("mortutil", IntegerType(), True),
    StructField("heqnum", IntegerType(), True),
    StructField("heqmof", IntegerType(), True),
    StructField("heqagbal", IntegerType(), True),
    StructField("hequtil", IntegerType(), True),
    StructField("derogpuh", IntegerType(), True),
    StructField("bankrpth", IntegerType(), True),
    StructField("bankn12h", IntegerType(), True),
    StructField("bankn24h", IntegerType(), True),
    StructField("agbkruph", IntegerType(), True),
    StructField("foreclhh", IntegerType(), True),
    StructField("forcl12h", IntegerType(), True),
    StructField("forcl24h", IntegerType(), True),
    StructField("lienthh", IntegerType(), True),
    StructField("lien12h", IntegerType(), True),
    StructField("lien24h", IntegerType(), True),
    StructField("rellienh", IntegerType(), True),
    StructField("satlienh", IntegerType(), True),
    StructField("chrgoffh", IntegerType(), True),
    StructField("chgof12h", IntegerType(), True),
    StructField("chgof24h", IntegerType(), True),
    StructField("agcobalh", IntegerType(), True),
    StructField("repothh", IntegerType(), True),
    StructField("repo12h", IntegerType(), True),
    StructField("repo24h", IntegerType(), True),
    StructField("judgsuih", IntegerType(), True),
    StructField("jdgst12h", IntegerType(), True),
    StructField("trustph", IntegerType(), True),
    StructField("numpubrh", IntegerType(), True),
    StructField("dprec12h", IntegerType(), True),
    StructField("dprec24h", IntegerType(), True),
    StructField("dprec36h", IntegerType(), True),
    StructField("dtaxlieh", IntegerType(), True),
    StructField("agpubreh", IntegerType(), True),
    StructField("agpub36h", IntegerType(), True),
    StructField("disbnkhh", IntegerType(), True),
    StructField("ndisbah", IntegerType(), True),
    StructField("bankpb6h", IntegerType(), True),
    StructField("bankrp6h", IntegerType(), True),
    StructField("paidjudh", IntegerType(), True),
    StructField("unpdjudh", IntegerType(), True),
    StructField("allcollh", IntegerType(), True),
    StructField("colexmeh", IntegerType(), True),
    StructField("collech", IntegerType(), True),
    StructField("colexm6h", IntegerType(), True),
    StructField("colle12h", IntegerType(), True),
    StructField("colxm12h", IntegerType(), True),
    StructField("colnm12h", IntegerType(), True),
    StructField("colle24h", IntegerType(), True),
    StructField("colxm24h", IntegerType(), True),
    StructField("colnm24h", IntegerType(), True),
    StructField("colle36h", IntegerType(), True),
    StructField("colxm36h", IntegerType(), True),
    StructField("colam24h", IntegerType(), True),
    StructField("colamt6h", IntegerType(), True),
    StructField("colam12h", IntegerType(), True),
    StructField("colam36h", IntegerType(), True),
    StructField("colamthh", IntegerType(), True),
    StructField("colbl24h", IntegerType(), True),
    StructField("colbal6h", IntegerType(), True),
    StructField("colbl12h", IntegerType(), True),
    StructField("colbl36h", IntegerType(), True),
    StructField("colcrlth", IntegerType(), True),
    StructField("paidcoh", IntegerType(), True),
    StructField("unpdcoh", IntegerType(), True),
    StructField("agcoltoh", IntegerType(), True),
    StructField("agcoltch", IntegerType(), True),
    StructField("retavgbl", IntegerType(), True),
    StructField("bkavgcl", IntegerType(), True),
    StructField("bkavgbal", IntegerType(), True),
    StructField("otrd01", IntegerType(), True),
    StructField("otrd02", IntegerType(), True),
    StructField("otrd03", IntegerType(), True),
    StructField("pubcp01", IntegerType(), True),
    StructField("coc001", IntegerType(), True),
    StructField("at03s", IntegerType(), True),
    StructField("at57s", IntegerType(), True),
    StructField("pbm003", IntegerType(), True),
    StructField("openbkc", IntegerType(), True),
    StructField("phone_number", StringType(), True),
    StructField("consumerid", StringType(), True),
    StructField("sequencenumber", StringType(), True),
    StructField("at01s", IntegerType(), True),
    StructField("at02s", IntegerType(), True),
    StructField("eads53_age", IntegerType(), True),
    StructField("at06s", IntegerType(), True),
    StructField("at09s", IntegerType(), True),
    StructField("at12s", IntegerType(), True),
    StructField("at20s", IntegerType(), True),
    StructField("at21s", IntegerType(), True),
    StructField("at24s", IntegerType(), True),
    StructField("at25s", IntegerType(), True),
    StructField("at27s", IntegerType(), True),
    StructField("at28a", IntegerType(), True),
    StructField("at28b", IntegerType(), True),
    StructField("at29s", IntegerType(), True),
    StructField("at30s", IntegerType(), True),
    StructField("at31s", IntegerType(), True),
    StructField("at32s", IntegerType(), True),
    StructField("at33a", IntegerType(), True),
    StructField("at33b", IntegerType(), True),
    StructField("at34a", IntegerType(), True),
    StructField("at34b", IntegerType(), True),
    StructField("at35a", IntegerType(), True),
    StructField("at35b", IntegerType(), True),
    StructField("at36s", IntegerType(), True),
    StructField("attr16", IntegerType(), True),
    StructField("at101s", IntegerType(), True),
    StructField("at101b", IntegerType(), True),
    StructField("at103s", IntegerType(), True),
    StructField("at104s", IntegerType(), True),
    StructField("au01s", IntegerType(), True),
    StructField("au02s", IntegerType(), True),
    StructField("au03s", IntegerType(), True),
    StructField("au06s", IntegerType(), True),
    StructField("au09s", IntegerType(), True),
    StructField("au12s", IntegerType(), True),
    StructField("au20s", IntegerType(), True),
    StructField("au21s", IntegerType(), True),
    StructField("au24s", IntegerType(), True),
    StructField("au25s", IntegerType(), True),
    StructField("au27s", IntegerType(), True),
    StructField("au28s", IntegerType(), True),
    StructField("au29s", IntegerType(), True),
    StructField("au30s", IntegerType(), True),
    StructField("au31s", IntegerType(), True),
    StructField("au32s", IntegerType(), True),
    StructField("au33s", IntegerType(), True),
    StructField("au34s", IntegerType(), True),
    StructField("au35s", IntegerType(), True),
    StructField("au36s", IntegerType(), True),
    StructField("au51a", IntegerType(), True),
    StructField("au57s", IntegerType(), True),
    StructField("au101s", IntegerType(), True),
    StructField("bc01s", IntegerType(), True),
    StructField("bc02s", IntegerType(), True),
    StructField("bc03s", IntegerType(), True),
    StructField("bc06s", IntegerType(), True),
    StructField("bc09s", IntegerType(), True),
    StructField("bc12s", IntegerType(), True),
    StructField("bc20s", IntegerType(), True),
    StructField("bc21s", IntegerType(), True),
    StructField("bc24s", IntegerType(), True),
    StructField("bc25s", IntegerType(), True),
    StructField("bc27s", IntegerType(), True),
    StructField("bc28s", IntegerType(), True),
    StructField("bc29s", IntegerType(), True),
    StructField("bc30s", IntegerType(), True),
    StructField("bc31s", IntegerType(), True),
    StructField("bc32s", IntegerType(), True),
    StructField("bc33s", IntegerType(), True),
    StructField("bc34s", IntegerType(), True),
    StructField("bc35s", IntegerType(), True),
    StructField("bc36s", IntegerType(), True),
    StructField("bc57s", IntegerType(), True),
    StructField("bc97a", IntegerType(), True),
    StructField("bc97b", IntegerType(), True),
    StructField("bc98a", IntegerType(), True),
    StructField("bc101s", IntegerType(), True),
    StructField("bc102s", IntegerType(), True),
    StructField("bc103s", IntegerType(), True),
    StructField("bc104s", IntegerType(), True),
    StructField("bc106s", IntegerType(), True),
    StructField("bc107s", IntegerType(), True),
    StructField("bc108s", IntegerType(), True),
    StructField("bc109s", IntegerType(), True),
    StructField("bc110s", IntegerType(), True),
    StructField("bi01s", IntegerType(), True),
    StructField("bi02s", IntegerType(), True),
    StructField("bi03s", IntegerType(), True),
    StructField("bi06s", IntegerType(), True),
    StructField("bi09s", IntegerType(), True),
    StructField("bi12s", IntegerType(), True),
    StructField("bi20s", IntegerType(), True),
    StructField("bi21s", IntegerType(), True),
    StructField("bi24s", IntegerType(), True),
    StructField("bi25s", IntegerType(), True),
    StructField("bi27s", IntegerType(), True),
    StructField("bi28s", IntegerType(), True),
    StructField("bi29s", IntegerType(), True),
    StructField("bi30s", IntegerType(), True),
    StructField("bi31s", IntegerType(), True),
    StructField("bi32s", IntegerType(), True),
    StructField("bi33s", IntegerType(), True),
    StructField("bi34s", IntegerType(), True),
    StructField("bi35s", IntegerType(), True),
    StructField("bi36s", IntegerType(), True),
    StructField("bi57s", IntegerType(), True),
    StructField("bi101s", IntegerType(), True),
    StructField("br01s", IntegerType(), True),
    StructField("br02s", IntegerType(), True),
    StructField("br03s", IntegerType(), True),
    StructField("br06s", IntegerType(), True),
    StructField("br09s", IntegerType(), True),
    StructField("br12s", IntegerType(), True),
    StructField("br20s", IntegerType(), True),
    StructField("br21s", IntegerType(), True),
    StructField("br24s", IntegerType(), True),
    StructField("br25s", IntegerType(), True),
    StructField("br27s", IntegerType(), True),
    StructField("br28s", IntegerType(), True),
    StructField("br29s", IntegerType(), True),
    StructField("br30s", IntegerType(), True),
    StructField("br31s", IntegerType(), True),
    StructField("br32s", IntegerType(), True),
    StructField("br33s", IntegerType(), True),
    StructField("br34s", IntegerType(), True),
    StructField("br35s", IntegerType(), True),
    StructField("br36s", IntegerType(), True),
    StructField("br57s", IntegerType(), True),
    StructField("br101s", IntegerType(), True),
    StructField("br109s", IntegerType(), True),
    StructField("co01s", IntegerType(), True),
    StructField("co02s", IntegerType(), True),
    StructField("co03s", IntegerType(), True),
    StructField("co04s", IntegerType(), True),
    StructField("co05s", IntegerType(), True),
    StructField("co06s", IntegerType(), True),
    StructField("co07s", IntegerType(), True),
    StructField("fc01s", IntegerType(), True),
    StructField("fc02s", IntegerType(), True),
    StructField("fc03s", IntegerType(), True),
    StructField("fc04s", IntegerType(), True),
    StructField("times_seen", StringType(), True),
    StructField("fc05s", IntegerType(), True),
    StructField("fc06s", IntegerType(), True),
    StructField("fc07s", IntegerType(), True),
    StructField("fi01s", IntegerType(), True),
    StructField("fi02s", IntegerType(), True),
    StructField("fi03s", IntegerType(), True),
    StructField("fi06s", IntegerType(), True),
    StructField("fi09s", IntegerType(), True),
    StructField("fi12s", IntegerType(), True),
    StructField("fi20s", IntegerType(), True),
    StructField("fi21s", IntegerType(), True),
    StructField("fi24s", IntegerType(), True),
    StructField("fi25s", IntegerType(), True),
    StructField("fi27s", IntegerType(), True),
    StructField("fi28s", IntegerType(), True),
    StructField("fi29s", IntegerType(), True),
    StructField("fi30s", IntegerType(), True),
    StructField("fi31s", IntegerType(), True),
    StructField("fi32s", IntegerType(), True),
    StructField("fi33s", IntegerType(), True),
    StructField("fi34s", IntegerType(), True),
    StructField("fi35s", IntegerType(), True),
    StructField("fi36s", IntegerType(), True),
    StructField("fi57s", IntegerType(), True),
    StructField("fi101s", IntegerType(), True),
    StructField("fr01s", IntegerType(), True),
    StructField("fr02s", IntegerType(), True),
    StructField("fr03s", IntegerType(), True),
    StructField("fr06s", IntegerType(), True),
    StructField("fr09s", IntegerType(), True),
    StructField("fr12s", IntegerType(), True),
    StructField("fr20s", IntegerType(), True),
    StructField("fr21s", IntegerType(), True),
    StructField("fr24s", IntegerType(), True),
    StructField("fr25s", IntegerType(), True),
    StructField("fr27s", IntegerType(), True),
    StructField("fr28s", IntegerType(), True),
    StructField("fr29s", IntegerType(), True),
    StructField("fr30s", IntegerType(), True),
    StructField("fr31s", IntegerType(), True),
    StructField("fr32s", IntegerType(), True),
    StructField("fr33s", IntegerType(), True),
    StructField("fr34s", IntegerType(), True),
    StructField("fr35s", IntegerType(), True),
    StructField("fr36s", IntegerType(), True),
    StructField("fr57s", IntegerType(), True),
    StructField("fr101s", IntegerType(), True),
    StructField("g001s", IntegerType(), True),
    StructField("g001b", IntegerType(), True),
    StructField("g001c", IntegerType(), True),
    StructField("g002s", IntegerType(), True),
    StructField("g002b", IntegerType(), True),
    StructField("g002c", IntegerType(), True),
    StructField("g003s", IntegerType(), True),
    StructField("g003c", IntegerType(), True),
    StructField("g020s", IntegerType(), True),
    StructField("g041s", IntegerType(), True),
    StructField("g041c", IntegerType(), True),
    StructField("g042s", IntegerType(), True),
    StructField("g042c", IntegerType(), True),
    StructField("g043s", IntegerType(), True),
    StructField("g043c", IntegerType(), True),
    StructField("g051s", IntegerType(), True),
    StructField("g057s", IntegerType(), True),
    StructField("g058s", IntegerType(), True),
    StructField("g059s", IntegerType(), True),
    StructField("g061s", IntegerType(), True),
    StructField("g063s", IntegerType(), True),
    StructField("g064s", IntegerType(), True),
    StructField("g066s", IntegerType(), True),
    StructField("g068s", IntegerType(), True),
    StructField("g069s", IntegerType(), True),
    StructField("g071s", IntegerType(), True),
    StructField("g093s", IntegerType(), True),
    StructField("g094s", IntegerType(), True),
    StructField("g095s", IntegerType(), True),
    StructField("g099s", IntegerType(), True),
    StructField("g100s", IntegerType(), True),
    StructField("g102s", IntegerType(), True),
    StructField("g103s", IntegerType(), True),
    StructField("g104s", IntegerType(), True),
    StructField("g105s", IntegerType(), True),
    StructField("g199s", IntegerType(), True),
    StructField("g200s", IntegerType(), True),
    StructField("g201a", IntegerType(), True),
    StructField("g201b", IntegerType(), True),
    StructField("g202a", IntegerType(), True),
    StructField("g202b", IntegerType(), True),
    StructField("g205s", IntegerType(), True),
    StructField("g206s", IntegerType(), True),
    StructField("g207s", IntegerType(), True),
    StructField("g208s", IntegerType(), True),
    StructField("g209s", IntegerType(), True),
    StructField("g210s", IntegerType(), True),
    StructField("g211s", IntegerType(), True),
    StructField("g212s", IntegerType(), True),
    StructField("g213a", IntegerType(), True),
    StructField("g213b", IntegerType(), True),
    StructField("g215a", IntegerType(), True),
    StructField("g215b", IntegerType(), True),
    StructField("g216s", IntegerType(), True),
    StructField("g217s", IntegerType(), True),
    StructField("g218a", IntegerType(), True),
    StructField("g218b", IntegerType(), True),
    StructField("g218c", IntegerType(), True),
    StructField("g218d", IntegerType(), True),
    StructField("g219a", IntegerType(), True),
    StructField("g219b", IntegerType(), True),
    StructField("g219c", IntegerType(), True),
    StructField("g219d", IntegerType(), True),
    StructField("g220a", IntegerType(), True),
    StructField("g220b", IntegerType(), True),
    StructField("g220c", IntegerType(), True),
    StructField("g220d", IntegerType(), True),
    StructField("g221a", IntegerType(), True),
    StructField("g221b", IntegerType(), True),
    StructField("g221c", IntegerType(), True),
    StructField("g221d", IntegerType(), True),
    StructField("g222s", IntegerType(), True),
    StructField("g223s", IntegerType(), True),
    StructField("g224a", IntegerType(), True),
    StructField("g224b", IntegerType(), True),
    StructField("g224c", IntegerType(), True),
    StructField("g225s", IntegerType(), True),
    StructField("g226s", IntegerType(), True),
    StructField("g227s", IntegerType(), True),
    StructField("g228s", IntegerType(), True),
    StructField("g230s", IntegerType(), True),
    StructField("g231s", IntegerType(), True),
    StructField("g232s", IntegerType(), True),
    StructField("g233s", IntegerType(), True),
    StructField("g234s", IntegerType(), True),
    StructField("g235s", IntegerType(), True),
    StructField("g236s", IntegerType(), True),
    StructField("g237s", IntegerType(), True),
    StructField("g238s", IntegerType(), True),
    StructField("g239s", IntegerType(), True),
    StructField("g240s", IntegerType(), True),
    StructField("g241s", IntegerType(), True),
    StructField("g250a", IntegerType(), True),
    StructField("g250b", IntegerType(), True),
    StructField("g250c", IntegerType(), True),
    StructField("g251a", IntegerType(), True),
    StructField("g251b", IntegerType(), True),
    StructField("g251c", IntegerType(), True),
    StructField("g300s", IntegerType(), True),
    StructField("g301s", IntegerType(), True),
    StructField("g302s", IntegerType(), True),
    StructField("g303s", IntegerType(), True),
    StructField("g304s", IntegerType(), True),
    StructField("g305s", IntegerType(), True),
    StructField("g306s", IntegerType(), True),
    StructField("g307s", IntegerType(), True),
    StructField("g308s", IntegerType(), True),
    StructField("g309s", IntegerType(), True),
    StructField("g310s", IntegerType(), True),
    StructField("g311s", IntegerType(), True),
    StructField("g312s", IntegerType(), True),
    StructField("g313s", IntegerType(), True),
    StructField("g314s", IntegerType(), True),
    StructField("g315s", IntegerType(), True),
    StructField("g960s", IntegerType(), True),
    StructField("g980s", IntegerType(), True),
    StructField("g990s", IntegerType(), True),
    StructField("hi01s", IntegerType(), True),
    StructField("hi02s", IntegerType(), True),
    StructField("hi03s", IntegerType(), True),
    StructField("hi06s", IntegerType(), True),
    StructField("hi09s", IntegerType(), True),
    StructField("hi12s", IntegerType(), True),
    StructField("hi20s", IntegerType(), True),
    StructField("hi21s", IntegerType(), True),
    StructField("hi24s", IntegerType(), True),
    StructField("hi25s", IntegerType(), True),
    StructField("hi27s", IntegerType(), True),
    StructField("hi28s", IntegerType(), True),
    StructField("hi29s", IntegerType(), True),
    StructField("hi30s", IntegerType(), True),
    StructField("hi31s", IntegerType(), True),
    StructField("hi32s", IntegerType(), True),
    StructField("hi33s", IntegerType(), True),
    StructField("hi34s", IntegerType(), True),
    StructField("hi35s", IntegerType(), True),
    StructField("hi36s", IntegerType(), True),
    StructField("hi57s", IntegerType(), True),
    StructField("hi101s", IntegerType(), True),
    StructField("hr01s", IntegerType(), True),
    StructField("hr02s", IntegerType(), True),
    StructField("hr03s", IntegerType(), True),
    StructField("hr06s", IntegerType(), True),
    StructField("hr09s", IntegerType(), True),
    StructField("hr12s", IntegerType(), True),
    StructField("hr20s", IntegerType(), True),
    StructField("hr21s", IntegerType(), True),
    StructField("hr24s", IntegerType(), True),
    StructField("hr25s", IntegerType(), True),
    StructField("hr27s", IntegerType(), True),
    StructField("hr28s", IntegerType(), True),
    StructField("hr29s", IntegerType(), True),
    StructField("hr30s", IntegerType(), True),
    StructField("hr31s", IntegerType(), True),
    StructField("hr32s", IntegerType(), True),
    StructField("hr33s", IntegerType(), True),
    StructField("hr34s", IntegerType(), True),
    StructField("hr35s", IntegerType(), True),
    StructField("hr36s", IntegerType(), True),
    StructField("hr57s", IntegerType(), True),
    StructField("hr101s", IntegerType(), True),
    StructField("in01s", IntegerType(), True),
    StructField("in02s", IntegerType(), True),
    StructField("in03s", IntegerType(), True),
    StructField("in06s", IntegerType(), True),
    StructField("in09s", IntegerType(), True),
    StructField("in12s", IntegerType(), True),
    StructField("in20s", IntegerType(), True),
    StructField("in21s", IntegerType(), True),
    StructField("in24s", IntegerType(), True),
    StructField("in25s", IntegerType(), True),
    StructField("in27s", IntegerType(), True),
    StructField("in28s", IntegerType(), True),
    StructField("in29s", IntegerType(), True),
    StructField("in30s", IntegerType(), True),
    StructField("in31s", IntegerType(), True),
    StructField("in32s", IntegerType(), True),
    StructField("in33s", IntegerType(), True),
    StructField("in34s", IntegerType(), True),
    StructField("in35s", IntegerType(), True),
    StructField("in36s", IntegerType(), True),
    StructField("in57s", IntegerType(), True),
    StructField("in101s", IntegerType(), True),
    StructField("lm01s", IntegerType(), True),
    StructField("lm02s", IntegerType(), True),
    StructField("lm03s", IntegerType(), True),
    StructField("lm04s", IntegerType(), True),
    StructField("lm05s", IntegerType(), True),
    StructField("lm06s", IntegerType(), True),
    StructField("lm07s", IntegerType(), True),
    StructField("lm08s", IntegerType(), True),
    StructField("lm09s", IntegerType(), True),
    StructField("lm10s", IntegerType(), True),
    StructField("lm11s", IntegerType(), True),
    StructField("lm12s", IntegerType(), True),
    StructField("lm13s", IntegerType(), True),
    StructField("lm14s", IntegerType(), True),
    StructField("lm15s", IntegerType(), True),
    StructField("lm16s", IntegerType(), True),
    StructField("lm17s", IntegerType(), True),
    StructField("lm18s", IntegerType(), True),
    StructField("lm19s", IntegerType(), True),
    StructField("lm20s", IntegerType(), True),
    StructField("lm21s", IntegerType(), True),
    StructField("lm22s", IntegerType(), True),
    StructField("lm23s", IntegerType(), True),
    StructField("lm24s", IntegerType(), True),
    StructField("lm25s", IntegerType(), True),
    StructField("lm26s", IntegerType(), True),
    StructField("lm27s", IntegerType(), True),
    StructField("lm28s", IntegerType(), True),
    StructField("lm29s", IntegerType(), True),
    StructField("lm30s", IntegerType(), True),
    StructField("lm31s", IntegerType(), True),
    StructField("lm32s", IntegerType(), True),
    StructField("lm33s", IntegerType(), True),
    StructField("lm34s", IntegerType(), True),
    StructField("lm35s", IntegerType(), True),
    StructField("lm36s", IntegerType(), True),
    StructField("lm37s", IntegerType(), True),
    StructField("lm38s", IntegerType(), True),
    StructField("lm39s", IntegerType(), True),
    StructField("mt01s", IntegerType(), True),
    StructField("mt02s", IntegerType(), True),
    StructField("mt03s", IntegerType(), True),
    StructField("mt06s", IntegerType(), True),
    StructField("mt09s", IntegerType(), True),
    StructField("mt12s", IntegerType(), True),
    StructField("mt20s", IntegerType(), True),
    StructField("mt21s", IntegerType(), True),
    StructField("mt24s", IntegerType(), True),
    StructField("mt25s", IntegerType(), True),
    StructField("mt27s", IntegerType(), True),
    StructField("mt28s", IntegerType(), True),
    StructField("mt29s", IntegerType(), True),
    StructField("mt30s", IntegerType(), True),
    StructField("mt31s", IntegerType(), True),
    StructField("mt32s", IntegerType(), True),
    StructField("mt33s", IntegerType(), True),
    StructField("mt34s", IntegerType(), True),
    StructField("mt34b", StringType(), True),
    StructField("mt34c", StringType(), True),
    StructField("mt35s", IntegerType(), True),
    StructField("mt36s", IntegerType(), True),
    StructField("mt47s", IntegerType(), True),
    StructField("mt57s", IntegerType(), True),
    StructField("mt101s", IntegerType(), True),
    StructField("md02s", IntegerType(), True),
    StructField("md33s", IntegerType(), True),
    StructField("md106s", IntegerType(), True),
    StructField("of01s", IntegerType(), True),
    StructField("of02s", IntegerType(), True),
    StructField("of03s", IntegerType(), True),
    StructField("of06s", IntegerType(), True),
    StructField("of09s", IntegerType(), True),
    StructField("of12s", IntegerType(), True),
    StructField("of20s", IntegerType(), True),
    StructField("of21s", IntegerType(), True),
    StructField("of24s", IntegerType(), True),
    StructField("of25s", IntegerType(), True),
    StructField("of27s", IntegerType(), True),
    StructField("of28s", IntegerType(), True),
    StructField("of29s", IntegerType(), True),
    StructField("of30s", IntegerType(), True),
    StructField("of31s", IntegerType(), True),
    StructField("of32s", IntegerType(), True),
    StructField("of33s", IntegerType(), True),
    StructField("of34s", IntegerType(), True),
    StructField("of35s", IntegerType(), True),
    StructField("of36s", IntegerType(), True),
    StructField("of57s", IntegerType(), True),
    StructField("of101s", IntegerType(), True),
    StructField("pb01s", IntegerType(), True),
    StructField("pb02s", IntegerType(), True),
    StructField("pb03s", IntegerType(), True),
    StructField("pb06s", IntegerType(), True),
    StructField("pb09s", IntegerType(), True),
    StructField("pb12s", IntegerType(), True),
    StructField("pb20s", IntegerType(), True),
    StructField("pb21s", IntegerType(), True),
    StructField("pb24s", IntegerType(), True),
    StructField("pb25s", IntegerType(), True),
    StructField("pb27s", IntegerType(), True),
    StructField("pb28s", IntegerType(), True),
    StructField("pb29s", IntegerType(), True),
    StructField("pb30s", IntegerType(), True),
    StructField("pb31s", IntegerType(), True),
    StructField("pb32s", IntegerType(), True),
    StructField("pb33s", IntegerType(), True),
    StructField("pb34s", IntegerType(), True),
    StructField("pb35s", IntegerType(), True),
    StructField("pb36s", IntegerType(), True),
    StructField("pb57s", IntegerType(), True),
    StructField("pb101s", IntegerType(), True),
    StructField("re01s", IntegerType(), True),
    StructField("re02s", IntegerType(), True),
    StructField("re03s", IntegerType(), True),
    StructField("re06s", IntegerType(), True),
    StructField("re09s", IntegerType(), True),
    StructField("re12s", IntegerType(), True),
    StructField("re20s", IntegerType(), True),
    StructField("re21s", IntegerType(), True),
    StructField("re24s", IntegerType(), True),
    StructField("re25s", IntegerType(), True),
    StructField("re27s", IntegerType(), True),
    StructField("re28s", IntegerType(), True),
    StructField("re29s", IntegerType(), True),
    StructField("re30s", IntegerType(), True),
    StructField("re31s", IntegerType(), True),
    StructField("re32s", IntegerType(), True),
    StructField("re33s", IntegerType(), True),
    StructField("re34s", IntegerType(), True),
    StructField("re35s", IntegerType(), True),
    StructField("re36s", IntegerType(), True),
    StructField("re57s", IntegerType(), True),
    StructField("re101s", IntegerType(), True),
    StructField("re102s", IntegerType(), True),
    StructField("rp01s", IntegerType(), True),
    StructField("rp02s", IntegerType(), True),
    StructField("rp03s", IntegerType(), True),
    StructField("rp04s", IntegerType(), True),
    StructField("rp05s", IntegerType(), True),
    StructField("rp06s", IntegerType(), True),
    StructField("rp07s", IntegerType(), True),
    StructField("rp08s", IntegerType(), True),
    StructField("rt01s", IntegerType(), True),
    StructField("rt02s", IntegerType(), True),
    StructField("rt03s", IntegerType(), True),
    StructField("rt06s", IntegerType(), True),
    StructField("rt09s", IntegerType(), True),
    StructField("rt12s", IntegerType(), True),
    StructField("rt20s", IntegerType(), True),
    StructField("rt21s", IntegerType(), True),
    StructField("rt24s", IntegerType(), True),
    StructField("rt25s", IntegerType(), True),
    StructField("rt27s", IntegerType(), True),
    StructField("rt28s", IntegerType(), True),
    StructField("rt29s", IntegerType(), True),
    StructField("rt30s", IntegerType(), True),
    StructField("rt31s", IntegerType(), True),
    StructField("rt32s", IntegerType(), True),
    StructField("rt33s", IntegerType(), True),
    StructField("rt34s", IntegerType(), True),
    StructField("rt35s", IntegerType(), True),
    StructField("rt36s", IntegerType(), True),
    StructField("rt57s", IntegerType(), True),
    StructField("rt101s", IntegerType(), True),
    StructField("rt201s", IntegerType(), True),
    StructField("s004s", IntegerType(), True),
    StructField("s043s", IntegerType(), True),
    StructField("s061s", IntegerType(), True),
    StructField("s062s", IntegerType(), True),
    StructField("s064a", IntegerType(), True),
    StructField("s064b", IntegerType(), True),
    StructField("s068a", IntegerType(), True),
    StructField("s068b", IntegerType(), True),
    StructField("s071a", IntegerType(), True),
    StructField("s071b", IntegerType(), True),
    StructField("s073a", IntegerType(), True),
    StructField("s073b", IntegerType(), True),
    StructField("s114s", IntegerType(), True),
    StructField("st01s", IntegerType(), True),
    StructField("st02s", IntegerType(), True),
    StructField("st03s", IntegerType(), True),
    StructField("st05s", IntegerType(), True),
    StructField("st06s", IntegerType(), True),
    StructField("st09s", IntegerType(), True),
    StructField("st12s", IntegerType(), True),
    StructField("st20s", IntegerType(), True),
    StructField("st21s", IntegerType(), True),
    StructField("st24s", IntegerType(), True),
    StructField("st25s", IntegerType(), True),
    StructField("st27s", IntegerType(), True),
    StructField("st28s", IntegerType(), True),
    StructField("st29s", IntegerType(), True),
    StructField("st30s", IntegerType(), True),
    StructField("st31s", IntegerType(), True),
    StructField("st32s", IntegerType(), True),
    StructField("st33s", IntegerType(), True),
    StructField("st34s", IntegerType(), True),
    StructField("st35s", IntegerType(), True),
    StructField("st36s", IntegerType(), True),
    StructField("st45s", IntegerType(), True),
    StructField("st50s", IntegerType(), True),
    StructField("st57s", IntegerType(), True),
    StructField("st60s", IntegerType(), True),
    StructField("st99s", IntegerType(), True),
    StructField("st101s", IntegerType(), True),
    StructField("atap01", IntegerType(), True),
    StructField("hiap01", IntegerType(), True),
    StructField("hrap01", IntegerType(), True),
    StructField("inap01", IntegerType(), True),
    StructField("mtap01", IntegerType(), True),
    StructField("reap01", IntegerType(), True),
    StructField("s063s", IntegerType(), True),
    StructField("s063a", IntegerType(), True),
    StructField("s208s", IntegerType(), True),
    StructField("s207s", IntegerType(), True),
    StructField("s207a", IntegerType(), True),
    StructField("g106s", IntegerType(), True),
    StructField("g099a", IntegerType(), True),
    StructField("s206s", IntegerType(), True),
    StructField("s209s", IntegerType(), True),
    StructField("s209a", IntegerType(), True),
    StructField("s204s", IntegerType(), True),
    StructField("s204a", IntegerType(), True),
    StructField("g242s", IntegerType(), True),
    StructField("g243s", IntegerType(), True),
    StructField("g244s", IntegerType(), True),
    StructField("g242f", IntegerType(), True),
    StructField("g243f", IntegerType(), True),
    StructField("g244f", IntegerType(), True),
    StructField("g242b", IntegerType(), True),
    StructField("g243b", IntegerType(), True),
    StructField("g244b", IntegerType(), True),
    StructField("st102s", IntegerType(), True),
    StructField("st103s", IntegerType(), True),
    StructField("at01a", IntegerType(), True),
    StructField("trd", IntegerType(), True),
    StructField("bc_trd", IntegerType(), True),
    StructField("inst_trd", IntegerType(), True),
    StructField("nomt_trd", IntegerType(), True),
    StructField("rev_trd", IntegerType(), True),
    StructField("rtl_trd", IntegerType(), True),
    StructField("agg101", IntegerType(), True),
    StructField("agg102", IntegerType(), True),
    StructField("agg103", IntegerType(), True),
    StructField("agg104", IntegerType(), True),
    StructField("agg105", IntegerType(), True),
    StructField("agg106", IntegerType(), True),
    StructField("agg107", IntegerType(), True),
    StructField("agg108", IntegerType(), True),
    StructField("agg109", IntegerType(), True),
    StructField("agg110", IntegerType(), True),
    StructField("agg111", IntegerType(), True),
    StructField("agg112", IntegerType(), True),
    StructField("agg113", IntegerType(), True),
    StructField("agg114", IntegerType(), True),
    StructField("agg115", IntegerType(), True),
    StructField("agg116", IntegerType(), True),
    StructField("agg117", IntegerType(), True),
    StructField("agg118", IntegerType(), True),
    StructField("agg119", IntegerType(), True),
    StructField("agg120", IntegerType(), True),
    StructField("agg121", IntegerType(), True),
    StructField("agg122", IntegerType(), True),
    StructField("agg123", IntegerType(), True),
    StructField("agg124", IntegerType(), True),
    StructField("agg201", IntegerType(), True),
    StructField("agg202", IntegerType(), True),
    StructField("agg203", IntegerType(), True),
    StructField("agg204", IntegerType(), True),
    StructField("agg205", IntegerType(), True),
    StructField("agg206", IntegerType(), True),
    StructField("agg207", IntegerType(), True),
    StructField("agg208", IntegerType(), True),
    StructField("agg209", IntegerType(), True),
    StructField("agg210", IntegerType(), True),
    StructField("agg211", IntegerType(), True),
    StructField("agg212", IntegerType(), True),
    StructField("agg213", IntegerType(), True),
    StructField("agg214", IntegerType(), True),
    StructField("agg215", IntegerType(), True),
    StructField("agg216", IntegerType(), True),
    StructField("agg217", IntegerType(), True),
    StructField("agg218", IntegerType(), True),
    StructField("agg219", IntegerType(), True),
    StructField("agg220", IntegerType(), True),
    StructField("agg221", IntegerType(), True),
    StructField("agg222", IntegerType(), True),
    StructField("agg223", IntegerType(), True),
    StructField("agg224", IntegerType(), True),
    StructField("agg301", IntegerType(), True),
    StructField("agg302", IntegerType(), True),
    StructField("agg303", IntegerType(), True),
    StructField("agg304", IntegerType(), True),
    StructField("agg305", IntegerType(), True),
    StructField("agg306", IntegerType(), True),
    StructField("agg307", IntegerType(), True),
    StructField("agg308", IntegerType(), True),
    StructField("agg309", IntegerType(), True),
    StructField("agg310", IntegerType(), True),
    StructField("agg311", IntegerType(), True),
    StructField("agg312", IntegerType(), True),
    StructField("agg313", IntegerType(), True),
    StructField("agg314", IntegerType(), True),
    StructField("agg315", IntegerType(), True),
    StructField("agg316", IntegerType(), True),
    StructField("agg317", IntegerType(), True),
    StructField("agg318", IntegerType(), True),
    StructField("agg319", IntegerType(), True),
    StructField("agg320", IntegerType(), True),
    StructField("agg321", IntegerType(), True),
    StructField("agg322", IntegerType(), True),
    StructField("agg323", IntegerType(), True),
    StructField("agg324", IntegerType(), True),
    StructField("agg401", IntegerType(), True),
    StructField("agg402", IntegerType(), True),
    StructField("agg403", IntegerType(), True),
    StructField("agg404", IntegerType(), True),
    StructField("agg405", IntegerType(), True),
    StructField("agg406", IntegerType(), True),
    StructField("agg407", IntegerType(), True),
    StructField("agg408", IntegerType(), True),
    StructField("agg409", IntegerType(), True),
    StructField("agg410", IntegerType(), True),
    StructField("agg411", IntegerType(), True),
    StructField("agg412", IntegerType(), True),
    StructField("agg413", IntegerType(), True),
    StructField("agg414", IntegerType(), True),
    StructField("agg415", IntegerType(), True),
    StructField("agg416", IntegerType(), True),
    StructField("agg417", IntegerType(), True),
    StructField("agg418", IntegerType(), True),
    StructField("agg419", IntegerType(), True),
    StructField("agg420", IntegerType(), True),
    StructField("agg421", IntegerType(), True),
    StructField("agg422", IntegerType(), True),
    StructField("agg423", IntegerType(), True),
    StructField("agg424", IntegerType(), True),
    StructField("agg501", IntegerType(), True),
    StructField("agg502", IntegerType(), True),
    StructField("agg503", IntegerType(), True),
    StructField("agg504", IntegerType(), True),
    StructField("agg505", IntegerType(), True),
    StructField("agg506", IntegerType(), True),
    StructField("agg507", IntegerType(), True),
    StructField("agg508", IntegerType(), True),
    StructField("agg509", IntegerType(), True),
    StructField("agg510", IntegerType(), True),
    StructField("agg511", IntegerType(), True),
    StructField("agg512", IntegerType(), True),
    StructField("agg513", IntegerType(), True),
    StructField("agg514", IntegerType(), True),
    StructField("agg515", IntegerType(), True),
    StructField("agg516", IntegerType(), True),
    StructField("agg517", IntegerType(), True),
    StructField("agg518", IntegerType(), True),
    StructField("agg519", IntegerType(), True),
    StructField("agg520", IntegerType(), True),
    StructField("agg521", IntegerType(), True),
    StructField("agg522", IntegerType(), True),
    StructField("agg523", IntegerType(), True),
    StructField("agg524", IntegerType(), True),
    StructField("agg601", IntegerType(), True),
    StructField("agg602", IntegerType(), True),
    StructField("agg603", IntegerType(), True),
    StructField("agg604", IntegerType(), True),
    StructField("agg605", IntegerType(), True),
    StructField("agg606", IntegerType(), True),
    StructField("agg607", IntegerType(), True),
    StructField("agg608", IntegerType(), True),
    StructField("agg609", IntegerType(), True),
    StructField("agg610", IntegerType(), True),
    StructField("agg611", IntegerType(), True),
    StructField("agg612", IntegerType(), True),
    StructField("agg613", IntegerType(), True),
    StructField("agg614", IntegerType(), True),
    StructField("agg615", IntegerType(), True),
    StructField("agg616", IntegerType(), True),
    StructField("agg617", IntegerType(), True),
    StructField("agg618", IntegerType(), True),
    StructField("agg619", IntegerType(), True),
    StructField("agg620", IntegerType(), True),
    StructField("agg621", IntegerType(), True),
    StructField("agg622", IntegerType(), True),
    StructField("agg623", IntegerType(), True),
    StructField("agg624", IntegerType(), True),
    StructField("agg701", IntegerType(), True),
    StructField("agg702", IntegerType(), True),
    StructField("agg703", IntegerType(), True),
    StructField("agg704", IntegerType(), True),
    StructField("agg705", IntegerType(), True),
    StructField("agg706", IntegerType(), True),
    StructField("agg707", IntegerType(), True),
    StructField("agg708", IntegerType(), True),
    StructField("agg709", IntegerType(), True),
    StructField("agg710", IntegerType(), True),
    StructField("agg711", IntegerType(), True),
    StructField("agg712", IntegerType(), True),
    StructField("agg713", IntegerType(), True),
    StructField("agg714", IntegerType(), True),
    StructField("agg715", IntegerType(), True),
    StructField("agg716", IntegerType(), True),
    StructField("agg717", IntegerType(), True),
    StructField("agg718", IntegerType(), True),
    StructField("agg719", IntegerType(), True),
    StructField("agg720", IntegerType(), True),
    StructField("agg721", IntegerType(), True),
    StructField("agg722", IntegerType(), True),
    StructField("agg723", IntegerType(), True),
    StructField("agg724", IntegerType(), True),
    StructField("agg801", IntegerType(), True),
    StructField("agg802", IntegerType(), True),
    StructField("agg803", IntegerType(), True),
    StructField("agg804", IntegerType(), True),
    StructField("agg805", IntegerType(), True),
    StructField("agg806", IntegerType(), True),
    StructField("agg807", IntegerType(), True),
    StructField("agg808", IntegerType(), True),
    StructField("agg809", IntegerType(), True),
    StructField("agg810", IntegerType(), True),
    StructField("agg811", IntegerType(), True),
    StructField("agg812", IntegerType(), True),
    StructField("agg813", IntegerType(), True),
    StructField("agg814", IntegerType(), True),
    StructField("agg815", IntegerType(), True),
    StructField("agg816", IntegerType(), True),
    StructField("agg817", IntegerType(), True),
    StructField("agg818", IntegerType(), True),
    StructField("agg819", IntegerType(), True),
    StructField("agg820", IntegerType(), True),
    StructField("agg821", IntegerType(), True),
    StructField("agg822", IntegerType(), True),
    StructField("agg823", IntegerType(), True),
    StructField("agg824", IntegerType(), True),
    StructField("agg901", IntegerType(), True),
    StructField("agg902", IntegerType(), True),
    StructField("agg903", IntegerType(), True),
    StructField("agg904", IntegerType(), True),
    StructField("agg905", StringType(), True),  # String field
    StructField("agg906", StringType(), True),  # String field
    StructField("agg907", IntegerType(), True),
    StructField("agg908", IntegerType(), True),
    StructField("agg909", IntegerType(), True),
    StructField("agg910", StringType(), True),  # String field
    StructField("agg911", StringType(), True),  # String field
    # trv fields
    *[StructField(f"trv{i:02}", IntegerType(), True) for i in range(1, 24)],

    # balmag fields
    StructField("balmag01", IntegerType(), True),
    StructField("balmag02", IntegerType(), True),

    # rvlr fields
    StructField("rvlr01", StringType(), True),
    StructField("rvlr02", StringType(), True),
    StructField("rvlr03", IntegerType(), True),
    StructField("rvlr04", IntegerType(), True),
    StructField("rvlr05", StringType(), True),
    StructField("rvlr06", StringType(), True),
    *[StructField(f"rvlr{i:02}", IntegerType() if i in range(7, 14) or i in range(29, 34) else StringType(), True)
      for i in range(7, 34)],

    # walshr fields
    *[StructField(f"walshr{i:02}", IntegerType() if i in range(1, 7) or i in range(9, 11) else StringType(), True)
      for i in range(1, 11)],

    # paymnt fields
    *[StructField(f"paymnt{i:02}", IntegerType() if i in range(1, 6) or i in range(10, 12) else StringType(), True)
      for i in range(1, 12)],

    # cv fields
    *[StructField(f"cv{i:02}", IntegerType(), True) for i in range(1, 31)],

    # rets fields
    *[StructField(f"rets{i:03}", IntegerType(), True) for i in list(range(101, 125)) + list(range(901, 906))],

    # rtdex fields
    *[StructField(f"rtdex{i:02}" if i < 3 else f"rtdexq{i - 2}", IntegerType(), True) for i in range(1, 7)],

    # walsrsrt fields
    *[StructField(f"walsrsrt{i}", IntegerType(), True) for i in range(1, 3)],

    # revs fields
    *[StructField(f"revs{i:03}", IntegerType(), True) for i in list(range(101, 125)) + list(range(901, 906))],

    # rvdex fields
    *[StructField(f"rvdex{i:02}" if i < 3 else f"rvdexq{i - 2}", IntegerType(), True) for i in range(1, 7)],

    # walsrs fields
    *[StructField(f"walsrs{i}", IntegerType(), True) for i in range(1, 3)],

    # aggs fields
    *[StructField(f"aggs{i:03}", IntegerType(), True) for i in list(range(101, 125)) + list(range(901, 906))],

    # index fields
    *[StructField(f"index{i}" if i < 3 else f"indexq{i - 2}", IntegerType(), True) for i in range(1, 7)],

    # walshrs fields
    *[StructField(f"walshrs{i}", IntegerType(), True) for i in range(1, 3)],

    # other numeric and string fields
    StructField("cvtg03", IntegerType(), True),
    StructField("pbn002", IntegerType(), True),
    StructField("pbn003", IntegerType(), True),
    StructField("pbn004", IntegerType(), True),
    StructField("pbn005", IntegerType(), True),
    StructField("gnni002", IntegerType(), True),
    StructField("bcpmtstr", StringType(), True),
    StructField("bcpmtnum", IntegerType(), True),

    # auc/aud/aum/aup/aur/aut fields (grouped by hundreds)
    *[
        StructField(f"{prefix}{i:04}", 
                    StringType() if prefix.startswith("aud") or prefix.startswith("aur") or prefix.startswith("aut") 
                    else IntegerType(),
                    True)
        for prefix in ["auc", "aud", "aum", "aup", "aur", "aut"]
        for group in [1000, 2000, 3000, 4000, 5000]
        for i in range(group + 1, group + (3 if prefix.startswith("aud") else (6 if prefix.startswith("aut") else 3)))
     ],

     # Additional numeric and string fields
     StructField("eusp01", IntegerType(), True),
     *[StructField(f"attr{i:02}", IntegerType(), True) for i in range(1, 5)],
     StructField("campaign_batch_id", StringType(), True),
     StructField("campaign_run_id", StringType(), True),
     StructField("etl_ingestion_time", StringType(), True),
     StructField("send_id", IntegerType(), True),

     # More descriptive numeric and string fields
     StructField("consecutive_mailings", IntegerType(), True),
     StructField("times_mailed_last_6_sends", IntegerType(), True),
     StructField("times_mailed_last_9_sends", IntegerType(), True),
     StructField("times_mailed_card_last_9_sends", IntegerType(), True),
     StructField("times_mailed_loan_last_6_sends", IntegerType(), True),
     StructField("times_mailed_loan_last_9_sends", IntegerType(), True),
     StructField("campaign_name", StringType(), True),

     # Decimal and string score-related fields
     *[
         StructField(field_name,
                     DecimalType(10,2) if field_name.endswith("_score") and not field_name.startswith(("pl_response_", "cc_response_")) 
                     else (StringType() if field_name.endswith("_score") else DecimalType()),
                     True)
         for field_name in [
             "income_300_score",
             "cc_short_term_110_score",
             "pl_short_term_100_score",
             "pl_response_400_score",
             "income_101_score",
             "bankruptcy_200_score",
             "default_510_xgb_score",
             "default_510_little_recent_score",
             "default_510_lots_recent_score",
             "cc_response_100_score",
             "credit_card_11_score",
             "credit_card_20_score",
             "cc_default_30_score",
             "cc_response_200_score",
             "cc_response_300_score",
             "pl_default_520_score"
         ]
      ],

      # Segmentation and metadata columns
      StructField("loan_risk_segment", StringType(), True),
      StructField("adj_pl_default_520_score", DecimalType(10,2),True ),
        StructField("product_rating", StringType(), True),
        StructField("ntile_100", IntegerType(), True),
        StructField("product_cd", StringType(), True),
        StructField("credit_policy", StringType(), True),
        StructField("inquiry_code", StringType(), True),
        StructField("eligibility_group_id", IntegerType(), True),
        StructField("desirability_group_id", IntegerType(), True),
        StructField("crossover_group_id", IntegerType(), True),
        StructField("priority", IntegerType(), True),
        StructField("row_count", IntegerType(), True),
        StructField("row_num", IntegerType(), True),
        StructField("creative_code", StringType(), True),
        StructField("transformation_config_id", IntegerType(), True),
        StructField("dedup_count", IntegerType(), True)

])


# List folders dynamically from S3
def list_s3_folders(bucket, prefix):
    paginator = s3.get_paginator("list_objects_v2")
    response_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix, Delimiter="/")
    folders = []
    for page in response_iterator:
        folders.extend([obj["Prefix"].split("/")[-2] for obj in page.get("CommonPrefixes", [])])
    return folders

folders = list_s3_folders(bucket_name, prefix)
if not folders:
    print("No folders found under the specified prefix.")
    exit()

print(f"Discovered Folders: {folders}")  # Debugging - Log detected folders


# Function to add missing columns based on schema
def align_to_schema(df, schema):
    for field in schema.fields:
        if field.name not in df.columns:
            df = df.withColumn(field.name, lit(None).cast(field.dataType))
    return df.select([col(field.name) for field in schema.fields])  # Reorder columns


import gc

def process_and_write(folder):
    input_path = f"s3://{bucket_name}/{prefix}{folder}/*.pq"
    output_folder_path = f"{output_path}{folder}/"  # Preserve folder structure in output

    try:
        # Read Parquet files without schema (schema inference)
        df = spark.read.parquet(input_path).repartition(10)  # Reduce partitions if necessary
        
        # Align DataFrame to the defined schema
        df_aligned = align_to_schema(df, schema)
        
        # Add source folder column
        df_aligned = df_aligned.withColumn("source_folder", lit(folder))
        
        # Write DataFrame back to S3 with the defined schema
        df_aligned.write.mode("overwrite").parquet(output_folder_path)
        print(f"Processed and written folder: {folder} -> {output_folder_path}")
        
        # Clear cache and force garbage collection
        spark.catalog.clearCache()
        del df, df_aligned  # Remove references to DataFrames
        gc.collect()  # Trigger garbage collection
    
    except AnalysisException as e:
        print(f"Skipping folder {folder} - {str(e)}")

# Process each folder sequentially
for folder in folders:
    process_and_write(folder)

print("All folders processed successfully.")




# import gc

# def process_and_write(folder):
#     input_path = f"s3://{bucket_name}/{prefix}{folder}/*.pq"
#     output_folder_path = f"{output_path}{folder}/"  # Preserve folder structure in output

#     try:
#         # Read Parquet files without schema (schema inference)
#         df = spark.read.parquet(input_path).repartition(10)  # Reduce partitions if necessary
        
#         # Align DataFrame to the defined schema
#         df_aligned = align_to_schema(df, schema)
        
#         # Add source folder column
#         df_aligned = df_aligned.withColumn("source_folder", lit(folder))
        
#         # Write DataFrame back to S3 with the defined schema
#         df_aligned.write.mode("overwrite").parquet(output_folder_path)
#         print(f"Processed and written folder: {folder} -> {output_folder_path}")
        
#         # Clear cache and force garbage collection
#         spark.catalog.clearCache()
#         del df, df_aligned  # Remove references to DataFrames
#         gc.collect()  # Trigger garbage collection
    
#     except AnalysisException as e:
#         print(f"Skipping folder {folder} - {str(e)}")

# # Process each folder in batches (optional)
# batch_size = 10  # Number of folders per batch

# for i in range(0, len(folders), batch_size):
#     batch_folders = folders[i:i + batch_size]
#     print(f"Processing batch: {batch_folders}")
    
#     for folder in batch_folders:
#         process_and_write(folder)


#     brea
# print("All folders processed successfully.")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o105.parquet.
: org.apache.spark.SparkException: Job 0 cancelled because killed via the Web UI
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2863)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:2745)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleStageCancellation$1(DAGScheduler.scala:2734)
	at scala.runtime.java8.JFunction1$mcVI$sp.apply(JFunction1$mcVI$sp.java:23)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:246)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:2727)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3007)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLo

In [ ]:
output_path

In [7]:
folders

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['10046', '10390', '10537', '10901', '1110', '11107', '11301', '11501', '11688', '11884', '12087', '1306', '137', '1406', '1567', '1822', '206', '2063', '2374', '2634', '2956', '320', '3268', '3570', '436', '4758', '5185', '5286', '5599', '576', '58', '5808', '6116', '6475', '6811', '7138', '731', '7490', '7671', '7963', '8239', '8604', '882', '8877', '9224', '9498', '9791']